In [80]:
import numpy as np
import pandas as pd
import skimage
import skimage.io as io
import cv2
import os
import pathlib
import random
import time

import imutils
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from skimage.color import rgb2gray,rgb2hsv
from skimage.filters import threshold_otsu
from skimage.filters import sobel
from skimage.exposure import histogram
from skimage import feature
from skimage.morphology import binary_erosion, binary_dilation, binary_closing, binary_opening
from __future__ import division
from scipy.signal import convolve2d
import numpy as np
import matplotlib.pyplot as plt
import joblib


In [81]:
PATH_TO_DATA ="./ACdata_base"

In [82]:
#Trials Block
def closingMinusImageRatio(image):
    image = np.asarray(image)
    closing = binary_closing(image)
    diff = closing - image
    whiteOriginal = np.sum(image[image == 1])
    whiteDiff = np.sum(diff[diff == 1])
    return whiteDiff / whiteOriginal


def getBaseline(image):
    img_row_sum = np.argmax(np.sum(image, axis=1))
    return img_row_sum


def largestComponentRatio(image):
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)
    sizes = stats[:, -1]
    countOfConnectedComp = len(output)
    # print(countOfConnectedComp)
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
    #show_images([image, img2])
    whiteAll = np.sum(image[image == 1])
    whiteComponent = np.sum(img2[img2 == 255])

    img2 = img2.astype('uint8')
    contours = cv2.findContours(img2, 1, 2)
    contours = sorted(contours[0], key=lambda x: cv2.contourArea(x), reverse=True)
    angle = cv2.minAreaRect(contours[0])[2]
    #print("angle: ", angle)

    return whiteComponent / (whiteAll * image.shape[0] * image.shape[1]), countOfConnectedComp, angle

def averageAngle(image):
    image = image.astype(np.uint8)
    contours = cv2.findContours(image, 1, 2)
    contours = sorted(contours[0], key=lambda x: cv2.contourArea(x), reverse=True)[:4]
    image2 = np.copy(image)
    angle = 0
    i = 0
    for cnt in contours:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        angle += rect[2]
        i +=1
        box = np.int0(box)
        #cv2.drawContours(image2, [box], 0, 1, 2)
    angle = angle/i
    #print("average angle: ", angle)
    #show_images([image, image2])
    return angle    


def getRatio(image, baseline, level):
    if level == -1:
        image_low = image[baseline:,:]
        white = np.sum(image_low[image_low == 1])
        black = image_low.shape[0] * image_low.shape[1] - white
    elif level == 1:
        image_high = image[:baseline,:]
        white = np.sum(image_high[image_high == 1])
        black = image_high.shape[0] * image_high.shape[1] - white
    else:
        white = np.sum(image[image == 1])
        black = image.shape[0] * image.shape[1] - white
    #print(white, black, image.shape[0] * image.shape[1])
    if black == 0:
        black = 0.00001
    ratio = white / black
    return ratio

def horizontalProjection(image):
    hProj = np.max(np.sum(image, axis = 1))
    return hProj
def verticalProjection(image):
    vProj = np.max(np.sum(image, axis = 0))
    return vProj
def HVLines(image):
    gray = rgb2gray(image)
    edges = cv2.Canny(image,200,100)
    linesVert = cv2.HoughLinesP(edges, 1, np.pi/180, 50, maxLineGap=250)
    linesHoriz = cv2.HoughLinesP(edges, 1, np.pi, 50, maxLineGap=250)
    if linesHoriz is None:
        linesHoriz=[]
    if linesVert is None:
        linesVert=[] 
    return len(linesVert),len(linesHoriz)

In [83]:
def lpq(img):
    rho=0.90
    winSize=3
    STFTalpha=1/winSize  
    img=np.float64(img) # Convert np.image to double
    radius=(winSize-1)/2 
    x=np.arange(-radius,radius+1)[np.newaxis] # Form spatial coordinates in window
    w0=np.ones_like(x)
    w1=np.exp(-2*np.pi*x*STFTalpha*1j)
    w2=np.conj(w1)

    ## Run filters to compute the frequency response in the four points
    filterResp1=convolve2d(convolve2d(img,w0.T,'valid'),w1,'valid')
    filterResp2=convolve2d(convolve2d(img,w1.T,'valid'),w0,'valid')
    filterResp3=convolve2d(convolve2d(img,w1.T,'valid'),w1,'valid')
    filterResp4=convolve2d(convolve2d(img,w1.T,'valid'),w2,'valid')

    # Initilize frequency domain matrix for four frequency coordinates
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

    #Perform quantization and compute LPQ codewords
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

    LPQdesc=np.histogram(LPQdesc.flatten(),range(100))[0]
    LPQdesc=LPQdesc/LPQdesc.sum()
    return LPQdesc

In [84]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

def get_accuracy(pred, Y):
    return round(sum(pred == Y) * 100 / float(len(Y)), 3)

In [85]:

def getBoundingBox(image):
    coords = cv2.findNonZero(image) # Find all non-zero points (text)
    x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
    rect = image[y:y+h, x:x+w]
    return rect


def unifyBackground(image):
    line1 = image[0,:]
    line2 = image[-1,:]
    avg1 = np.average(line1)
    avg2 = np.average(line2)
    avg = (avg1 + avg2) / 2
    if avg > 0.5:
        image = 1-image
    return image


def denoise(image):
    # Find contours and remove small noise
    img=np.copy(image)
    img=image.astype(np.uint8)
    cnts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    rect_areas = []
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        rect_areas.append(w * h)
    avg_area = np.mean(rect_areas)
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        cnt_area = w * h
        if cnt_area < 0.01 * avg_area:
            img[y:y + h, x:x + w] = 0          
    return img        


def preprocessing(image):
    gray = rgb2gray(image)
    # gray=cv2.resize(gray,(400,200))
    thresh = threshold_otsu(gray)
    gray[gray > thresh] = 1
    gray[gray < thresh] = 0
    gray = unifyBackground(gray)
    denoised= denoise(gray)
    gray = getBoundingBox(denoised)
    return gray


In [119]:
image_test = []
X_test = []
Y_test = []

trainImage = []
labels = []
features = []
test_cases = os.listdir(PATH_TO_DATA)
test_cases.sort()
for font in test_cases:
        for image in pathlib.Path(PATH_TO_DATA + '/' + font).iterdir():
            trainImage.append(image)
            labels.append(float(font))

for image in trainImage:
    img = cv2.imread(str(image))
    preprocessedImg = preprocessing(img)
    lp=lpq(preprocessedImg)
    features.append(lp)

features = np.asarray(features)

labels = np.asarray(labels)
#X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2,random_state=101) # 80% training and 20% test

In [123]:

X_train = features
Y_train = labels

# clf = MLPClassifier(random_state=1, max_iter=200 , hidden_layer_sizes=(255, 1000))
# clf2 = DecisionTreeClassifier(max_depth=10, random_state=1)
# clf3 = KNeighborsClassifier(n_neighbors=3)
clf4= RandomForestClassifier(max_depth=25, random_state=1)
# clf5= svm.SVC(gamma=20,C=200)

        
#Train the model using the training sets
# clf.fit(X_train, Y_train)
# clf2.fit(X_train, Y_train)
# clf3.fit(X_train, Y_train)
clf4.fit(X_train,Y_train)
# clf5.fit(X_train,Y_train)

RandomForestClassifier(max_depth=25, random_state=1)

In [125]:
#export random forest module
filename = "Completed_model.joblib"
joblib.dump(clf4, filename)


#Predict the response for test dataset
# pred=clf.predict_proba(X_test)
# correct=0
# j=0
# for i in pred:
#     a=np.argmax(i)
#     a=float(a)+1
#     if (a)==Y_test[j]:
#         correct=correct+1
#     j=j+1

# pred_test2 = clf2.predict(X_test)
# pred_test3 = clf3.predict(X_test)
# pred_test4 = clf4.predict(X_test)
# pred_test5 = clf5.predict(X_test)


# print("MLP:", correct*100/len(Y_test))
# print("Decision Tree: ", (pred_test2))
# print("KNN: ", (pred_test3))
# print("Random Forrest: " ,(pred_test4))
# print("SVM: " ,(pred_test5))


['Completed_model.joblib']